In [2]:
import httpx
import time
import pandas as pd

In [83]:
HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "nova"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
    "sort": "new",
    "limit": 25,
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

environmental_keywords = ["pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["electricity", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "approval", "permit", "city council"]
# NEED TO INCLUDE THIS FILTERING LATER, PROB WHEN I HAVE MORE DATA WHICH IS LIKE NOW
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    r = client.get(search_url, params=params)
    r.raise_for_status()
    listing = r.json()

    for child in listing["data"]["children"]:
        matched = False

        if any(kw in child["data"]["selftext"].lower() for kw in environmental_keywords):
            env_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in infrastructure_utilities_keywords):
            infr_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in housing_keywords):
            housing_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in economic_keywords):
            econ_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in life_quality_keywords):
            life_qual_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in aesthetics_keywords):
            aesth_post_ids.append(child["data"]["id"])
            matched = True
        if any(kw in child["data"]["selftext"].lower() for kw in government_keywords):
            gov_post_ids.append(child["data"]["id"])
            matched = True
        if not matched:
            not_useful_post_ids.append(child["data"]["id"])

        time.sleep(0.2)

print("Found environmental posts:", env_post_ids)
print("Found infrastructure posts:", infr_post_ids)
print("Found housing posts:", housing_post_ids)
print("Found economic posts:", econ_post_ids)
print("Found life quality posts:", life_qual_post_ids)
print("Found aesthetic posts:", aesth_post_ids)
print("Found government posts:", gov_post_ids)

# Fetch each post's JSON
# posts_json = []
# with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
#     for pid in housing_post_ids:
#         post_url = f"https://www.reddit.com/comments/{pid}.json"
#         r = client.get(post_url)
#         r.raise_for_status()
#         posts_json.append(r.json())
#         listing = r.json()
        
#         time.sleep(1)  # important: be polite

# print("Fetched", len(posts_json), "posts")

Found environmental posts: ['7htslh']
Found infrastructure posts: ['1gawpld', 'wktuak', 'eddta7', '7htslh']
Found housing posts: ['1edvmw5', '15cwyr6', '158wywd', '152m2sz', 'j25zf4', '7htslh']
Found economic posts: ['1ljjzo8', '15cwyr6', 'wktuak', 'eddta7']
Found life quality posts: ['15cwyr6', '158wywd', '152m2sz', '1341j3h', 'j25zf4', 'eddta7']
Found aesthetic posts: ['1lko81m']
Found government posts: ['1341j3h']


In [84]:
MAX_POSTS = 500

HEADERS = {
    "User-Agent": "research-script/0.1 (academic, non-commercial)"
}

subreddit = "nova"
search_url = f"https://www.reddit.com/r/{subreddit}/search.json"

params = {
    "q": 'datacenter',
    "restrict_sr": 1,
    "sort": "new",
    "t": "all",
    "limit": 25,
}

env_post_ids = []
infr_post_ids = []
housing_post_ids = []
econ_post_ids = []
life_qual_post_ids = []
aesth_post_ids = []
gov_post_ids = []
not_useful_post_ids = []

after = None

environmental_keywords = ["pollution", "emissions", "diesel", "fumes", "light pollution", "water", "drought", "water consumption",  "waste", "carbon", "climate"]
infrastructure_utilities_keywords = ["electricity", "power", "grid", "blackout", "substation", "reliable", "reliability", "capacity", "infrastructure", "generator", "water"]
housing_keywords = ["housing", "house", "home", "rent", "property", "real estate", "house price", "housing price", "housing cost", "land cost", "land price", "gentrification"]
economic_keywords = ["job", "work", "revenue", "tax", "money", "rich", "salary", "economy", "investment", "business"]
life_quality_keywords = ["resident", "fumes", "noise", "loud", "vibration", "light pollution", "sound", "smell", "traffic"]
aesthetics_keywords = ["landscape", "visual impact",  "aesthetics", "looks", "architecture", "design", "facade", "industrial", "windows", "dark"]
government_keywords = ["zoning", "planning", "approval", "permit", "city council"]
# NEED TO INCLUDE THIS FILTERING LATER, PROB WHEN I HAVE MORE DATA WHICH IS LIKE NOW
# dont_include = ["server", "rack", "switch", "firewall", "BGP", "latency"]

with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
    while True:
        if after is not None:
            params["after"] = after
        else:
            params.pop("after", None)

        r = client.get(search_url, params=params)
        r.raise_for_status()
        listing = r.json()

        children = listing["data"]["children"]
        if not children:
            break

        for child in children:
            data = child.get("data", {})
            post_id = data.get("id")

            title = (data.get("title") or "")
            selftext = (data.get("selftext") or "")
            text = (title + "\n" + selftext).lower()

            matched = False
        
            if any(kw in text for kw in environmental_keywords):
                env_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in infrastructure_utilities_keywords):
                infr_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in housing_keywords):
                housing_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in economic_keywords):
                econ_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in life_quality_keywords):
                life_qual_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in aesthetics_keywords):
                aesth_post_ids.append(post_id)
                matched = True
            if any(kw in text for kw in government_keywords):
                gov_post_ids.append(post_id)
                matched = True
            if not matched:
                not_useful_post_ids.append(post_id)
        
        after = listing["data"]["after"]
        if after is None:
            break

        if len(post_ids) >= MAX_POSTS:
            break

        time.sleep(0.2)

def dedupe(xs):
    return list(dict.fromkeys([x for x in xs if x]))

env_post_ids = dedupe(env_post_ids)
infr_post_ids = dedupe(infr_post_ids)
housing_post_ids = dedupe(housing_post_ids)
econ_post_ids = dedupe(econ_post_ids)
life_qual_post_ids = dedupe(life_qual_post_ids)
aesth_post_ids = dedupe(aesth_post_ids)
gov_post_ids = dedupe(gov_post_ids)
not_useful_post_ids = dedupe(not_useful_post_ids)

print("Found environmental posts:", len(env_post_ids))
print("Found infrastructure posts:", len(infr_post_ids))
print("Found housing posts:", len(housing_post_ids))
print("Found economic posts:", len(econ_post_ids))
print("Found life quality posts:", len(life_qual_post_ids))
print("Found aesthetic posts:", len(aesth_post_ids))
print("Found government posts:", len(gov_post_ids))

Found environmental posts: 1
Found infrastructure posts: 5
Found housing posts: 6
Found economic posts: 4
Found life quality posts: 6
Found aesthetic posts: 1
Found government posts: 1


In [85]:
print(posts_json)

#listing -> data -> children (only 1 when going to a specific post) -> data -> selftext/created/id
#comments:children - data - body

#comments have to be above 200 characters? - some that are under express good opinions so idk (maybe over 75?)

[[{'kind': 'Listing', 'data': {'after': None, 'dist': 1, 'modhash': '', 'geo_filter': '', 'children': [{'kind': 't3', 'data': {'approved_at_utc': None, 'subreddit': 'nova', 'selftext': "My spouse and I almost toured a home that we later found out would back right up to these behemoths, assuming this massive project gets approved (and I can't imagine why it won't go through at this point).\n\nThe owners cancelled our tour the night before we were to show up in the morning and pulled their home off the market.\n\nSometimes you get lucky without even realizing it at first.", 'user_reports': [], 'saved': False, 'mod_reason_title': None, 'gilded': 0, 'clicked': False, 'title': 'Saved from the PW Digital Gateway without knowing it', 'link_flair_richtext': [], 'subreddit_name_prefixed': 'r/nova', 'hidden': False, 'pwls': 7, 'link_flair_css_class': None, 'downs': 0, 'thumbnail_height': 140, 'top_awarded_type': None, 'hide_score': False, 'name': 't3_1edvmw5', 'quarantine': False, 'link_flair_te

In [21]:
nova_theme_lists = [env_post_ids, infr_post_ids, housing_post_ids, econ_post_ids, life_qual_post_ids, aesth_post_ids, gov_post_ids]
posts = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])

for theme in nova_theme_lists:
    df1 = pd.DataFrame(columns=["ids", "text", "date", "environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"])
    post_ids = []
    post_texts = []
    post_dates = []
    with httpx.Client(headers=HEADERS, follow_redirects=True) as client:
        for pid in theme:
            post_url = f"https://www.reddit.com/comments/{pid}.json"                
            r = client.get(post_url)
            r.raise_for_status()
            post_json = r.json()
            post_ids.append(pid)
            post_texts.append(post_json[0]["data"]["children"][0]["data"]["selftext"])
            post_dates.append(post_json[0]["data"]["children"][0]["data"]["created_utc"])
            time.sleep(0.2)
    df1["ids"] = post_ids
    df1["text"] = post_texts
    df1["date"] = post_dates
    if theme == env_post_ids:
        df1["environment"] = True
    if theme == infr_post_ids:
        df1["infrastructure"] = True
    if theme == housing_post_ids:
        df1["housing"] = True
    if theme == econ_post_ids:
        df1["economy"] = True
    if theme == life_qual_post_ids:
        df1["life quality"] = True
    if theme == aesth_post_ids:
        df1["aesthetics"] = True
    if theme == gov_post_ids:
        df1["government"] = True
    posts = pd.concat([posts, df1], ignore_index=True)


print(posts)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_8530/3953131038.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  posts = pd.concat([posts, df1], ignore_index=True)


        ids                                               text          date  \
0    7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
1   1gawpld  Is it time to to slow down Datacenter growth a...  1.729754e+09   
2    wktuak  It seems there's a solid possibility I will ge...  1.660127e+09   
3    eddta7  Everybody in this area knows that datacenters ...  1.576866e+09   
4    7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
5   1edvmw5  My spouse and I almost toured a home that we l...  1.722129e+09   
6   15cwyr6  I feel like I hear lots of whining from Loudon...  1.690649e+09   
7   158wywd  \n\nOff loco pkwy before the greenway. Hope no...  1.690257e+09   
8   152m2sz  Citing Loudoun County property records, [BizJo...  1.689648e+09   
9    j25zf4  [https://www.bizjournals.com/washington/news/2...  1.601407e+09   
10   7htslh  Odd one of the day.\n\nJust me fishing. I have...  1.512516e+09   
11  1ljjzo8  I don't know what else to d

In [61]:
grouping_cols = ["ids", "text", "date"]
posts = posts.groupby(grouping_cols, as_index=False).max()
theme_cols = ["environment", "infrastructure", "housing", "economy", "life quality", "aesthetics", "government"]
posts[theme_cols] = posts[theme_cols].fillna(False)
posts.head(15)

/var/folders/9m/h28gbbc970j03ncf7v7dhqk80000gq/T/ipykernel_8530/3906843801.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  posts[theme_cols] = posts[theme_cols].fillna(False)


,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government,label,score,degree
0,1341j3h,"Hi, I'm a local youtuber planning on making a ...",1.682885e+09,False,False,False,False,False,False,True,neutral,0.765928,0.765928
1,152m2sz,"Citing Loudoun County property records, [BizJo...",1.689648e+09,False,False,True,False,True,False,False,neutral,0.836653,0.836653
2,158wywd,\n\nOff loco pkwy before the greenway. Hope no...,1.690257e+09,False,False,True,False,False,False,False,positive,0.809821,0.809821
3,15cwyr6,I feel like I hear lots of whining from Loudon...,1.690649e+09,False,False,True,True,True,False,False,negative,0.587637,0.587637
4,1edvmw5,My spouse and I almost toured a home that we l...,1.722129e+09,False,False,True,False,False,False,False,neutral,0.448640,0.448640
5,1gawpld,Is it time to to slow down Datacenter growth a...,1.729754e+09,False,True,False,False,False,False,False,neutral,0.586820,0.586820
6,1ljjzo8,I don't know what else to do anymore. For the ...,1.750792e+09,False,False,False,True,False,False,False,negative,0.816951,0.816951
7,1lko81m,I absolutely loathe these buildings. The strip...,1.750903e+09,False,False,False,False,False,True,False,negative,0.962419,0.962419
8,7htslh,Odd one of the day.\n\nJust me fishing. I have...,1.512516e+09,True,True,True,False,False,False,False,neutral,0.661281,0.661281
9,eddta7,Everybody in this area knows that datacenters ...,1.576866e+09,False,True,False,True,True,False,False,negative,0.462182,0.462182


In [62]:
import re
from transformers import pipeline
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment",
    truncation=True,      # <--- important
    padding=True,         # <--- for batching
    max_length=512        
)

Device set to use mps:0


In [63]:
texts = posts["text"].astype(str).tolist()
results = classifier(texts, truncation=True, padding=True, max_length=512, batch_size=32)
posts["roberta"] = results

In [64]:
posts["name"] = posts["roberta"].apply(lambda x: x["label"])

def roberta_label(name):
    if name == 'LABEL_2':
        return "positive"
    elif name == 'LABEL_1':
        return "neutral"
    else:
        return "negative"
posts["label"] = posts["name"].apply(roberta_label)

posts["degree"] = posts["roberta"].apply(lambda x: x["score"])
posts = posts.drop(["roberta", "name"], axis=1)

In [65]:
posts.head(15)

,ids,text,date,environment,infrastructure,housing,economy,life quality,aesthetics,government,label,score,degree
0,1341j3h,"Hi, I'm a local youtuber planning on making a ...",1.682885e+09,False,False,False,False,False,False,True,neutral,0.765928,0.765928
1,152m2sz,"Citing Loudoun County property records, [BizJo...",1.689648e+09,False,False,True,False,True,False,False,neutral,0.836653,0.836653
2,158wywd,\n\nOff loco pkwy before the greenway. Hope no...,1.690257e+09,False,False,True,False,False,False,False,positive,0.809821,0.809821
3,15cwyr6,I feel like I hear lots of whining from Loudon...,1.690649e+09,False,False,True,True,True,False,False,negative,0.587637,0.587637
4,1edvmw5,My spouse and I almost toured a home that we l...,1.722129e+09,False,False,True,False,False,False,False,neutral,0.448640,0.448640
5,1gawpld,Is it time to to slow down Datacenter growth a...,1.729754e+09,False,True,False,False,False,False,False,neutral,0.586820,0.586820
6,1ljjzo8,I don't know what else to do anymore. For the ...,1.750792e+09,False,False,False,True,False,False,False,negative,0.816951,0.816951
7,1lko81m,I absolutely loathe these buildings. The strip...,1.750903e+09,False,False,False,False,False,True,False,negative,0.962419,0.962419
8,7htslh,Odd one of the day.\n\nJust me fishing. I have...,1.512516e+09,True,True,True,False,False,False,False,neutral,0.661281,0.661281
9,eddta7,Everybody in this area knows that datacenters ...,1.576866e+09,False,True,False,True,True,False,False,negative,0.462182,0.462182


In [71]:
# calculating average sentiment based on theme:

def avg_sentiment_calculation(theme):
    theme_posts = posts[posts[theme] == True]
    avg_sentiment = theme_posts.loc[theme_posts["label"] == "positive", "score"].sum() - theme_posts.loc[theme_posts["label"] == "negative", "score"].sum()
    return avg_sentiment/len(theme_posts)

print("Average sentiment of environmental posts:", avg_sentiment_calculation("environment"))
print("Average sentiment of infrastructural posts:", avg_sentiment_calculation("infrastructure"))
print("Average sentiment of housing-related posts:", avg_sentiment_calculation("housing"))
print("Average sentiment of economic posts:", avg_sentiment_calculation("economy"))
print("Average sentiment of life-quality-related posts:", avg_sentiment_calculation("life quality"))
print("Average sentiment of aesthetics-related posts:", avg_sentiment_calculation("aesthetics"))
print("Average sentiment of governmental posts:", avg_sentiment_calculation("government"))

Average sentiment of environmental posts: 0.0
Average sentiment of infrastructural posts: -0.24285781383514404
Average sentiment of housing-related posts: 0.13187782963116965
Average sentiment of economic posts: -0.5940049886703491
Average sentiment of life-quality-related posts: -0.1201840490102768
Average sentiment of aesthetics-related posts: -0.9624190330505371
Average sentiment of governmental posts: 0.0
